# Determining the dominant eye of a viewer

1) How can we determine the dominant eye of a viewer? Can we say that for one image, the dominant eye is the one having the most fixation points? Is that consistent over all images for one viewer? $\rightarrow$ stats for every viewer
2) If we choose one fixation point to pass SAM as an input, should it be the point with the longest duration or the last point?

In [9]:
import pandas as pd
import os
import numpy as np

In [5]:
DIR = "/kaggle/input/df-pkl"
df = pd.read_pickle(os.path.join(DIR, "eye_tracking_data.pkl"))

df.head()

,class,image_file,width_height,offset,viewer,response,image_coordinates_left,image_start_end_time_left,image_duration_left,image_coordinates_right,image_start_end_time_right,image_duration_right,screen_coordinates_left,screen_coordinates_right
0,diningtable,2010_001366,"[332, 500]","[715, 279, 1215, 611]",1,1,"[(111.79999999999995, 256.79999999999995), (31...","[(7, 149), (186, 331), (359, 603), (618, 991),...","[142, 145, 244, 373, 514, 436]","[(100.79999999999995, 260.1), (304.70000000000...","[(7, 147), (184, 331), (357, 600), (616, 990),...","[140, 147, 243, 374, 517, 436]","[(826.8, 535.8), (1029.8, 503.8), (961.3, 446....","[(815.8, 539.1), (1019.7, 508.2), (947.9, 446...."
1,diningtable,2010_001366,"[332, 500]","[715, 279, 1215, 611]",2,1,"[(nan, nan), (318.70000000000005, 253.10000000...","[(nan, nan), (802.0, 1165.0), (1197.0, 1654.0)...","[nan, 363.0, 457.0, 291.0, 261.0]","[(213.79999999999995, 194.39999999999998), (31...","[(66, 767), (804, 1166), (1196, 1654), (1689, ...","[701, 362, 458, 289, 265]","[(nan, nan), (1033.7, 532.1), (977.1, 514.1), ...","[(928.8, 473.4), (1025.1, 525.2), (968.7, 507...."
2,diningtable,2010_001366,"[332, 500]","[715, 279, 1215, 611]",3,1,"[(163.20000000000005, 238.60000000000002), (14...","[(7, 105), (123, 265), (301, 582), (605, 993),...","[98, 142, 281, 388, 230]","[(170.5, 233.70000000000005), (153.20000000000...","[(7, 108), (120, 265), (296, 583), (606, 995),...","[101, 145, 287, 389, 229]","[(878.2, 517.6), (858.9, 517.1), (940.8, 435.1...","[(885.5, 512.7), (868.2, 507.0), (945.7, 417.1..."
3,diningtable,2010_001366,"[332, 500]","[715, 279, 1215, 611]",4,1,"[(228.29999999999995, 272.0), (234.20000000000...","[(16, 281), (317, 568), (589, 1115)]","[265, 251, 526]","[(214.89999999999998, 253.89999999999998), (22...","[(15, 283), (305, 568), (587, 1115)]","[268, 263, 528]","[(943.3, 551.0), (949.2, 482.6), (943.7, 527.0)]","[(929.9, 532.9), (937.2, 457.3), (934.3, 505.8)]"
4,diningtable,2010_001366,"[332, 500]","[715, 279, 1215, 611]",5,1,"[(139.29999999999995, 211.7), (218.10000000000...","[(7, 227), (268, 441), (464, 894), (1016, 1741)]","[220, 173, 430, 725]","[(153.60000000000002, 218.39999999999998), (24...","[(7, 227), (270, 440), (464, 894), (1014, 1741)]","[220, 170, 430, 727]","[(854.3, 490.7), (933.1, 406.6), (937.3, 476.0...","[(868.6, 497.4), (955.5, 425.7), (966.7, 475.8..."


### Idea 1: The dominant eye is the eye with the most fixation points per image. 

In [8]:
def count_fixation_points(coord_column):
    return coord_column.apply(lambda x: len([pt for pt in x if not (pd.isna(pt[0]) or pd.isna(pt[1]))]))

# Count fixation points for left and right eyes
df['left_count'] = count_fixation_points(df['image_coordinates_left'])
df['right_count'] = count_fixation_points(df['image_coordinates_right'])

# Group by viewer and sum the counts
viewer_totals = df.groupby('viewer')[['left_count', 'right_count']].sum()

# Calculate percentages for each viewer
viewer_totals['left_percentage'] = (viewer_totals['left_count'] / 
                                    (viewer_totals['left_count'] + viewer_totals['right_count'])) * 100
viewer_totals['right_percentage'] = (viewer_totals['right_count'] / 
                                     (viewer_totals['left_count'] + viewer_totals['right_count'])) * 100

# Add a dominant eye column
viewer_totals['dominant_eye'] = np.where(viewer_totals['left_percentage'] > viewer_totals['right_percentage'], 
                                         'Left', 'Right')

# Display results
print(viewer_totals[['left_percentage', 'right_percentage', 'dominant_eye']])

        left_percentage  right_percentage dominant_eye
viewer                                                
1             48.482869         51.517131        Right
2             45.112239         54.887761        Right
3             47.327483         52.672517        Right
4             48.346837         51.653163        Right
5             48.992510         51.007490        Right


### Idea 2: The dominant eye is the more exact eye (where the point of the longest duration lies on the object)

$\rightarrow$ needs visual investigation. Has been done in analysis-of-fixation-patterns.ipynb